<a href="https://colab.research.google.com/github/hchaparov/Dynamic_Pricing_MARL/blob/main/Altern_2A_PONG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install swig
!pip install "pettingzoo"
!pip install "stable-baselines3"
!pip install "supersuit"
!pip install pymunk
!pip install pyvirtualdisplay imageio[ffmpeg]
!apt-get install -y xvfb
!pip install tensorboard
!pip install pillow
!pip install multi_agent_ale_py
!pip install autorom
!pip install moviepy
!pip install tensorflow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (524 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [2]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms
	/usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/adventure.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/air_raid.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/alien.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/amidar.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/assault.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_

In [3]:
from __future__ import annotations

from pyvirtualdisplay import Display
import imageio
import numpy as np
import glob
import os
import time
import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy
from pettingzoo.atari import pong_v3
from PIL import Image
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, BaseCallback
from stable_baselines3.common.logger import configure
from moviepy.editor import ImageSequenceClip
from stable_baselines3.common.vec_env import VecVideoRecorder
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback
from google.colab import drive
drive.mount('/content/drive')

log_dir = "/content/drive/MyDrive/2A_PONG_ppo/"
os.makedirs(log_dir, exist_ok=True)

tensorboard_log_dir = "/content/drive/MyDrive/2A_PONG_ppo/tensorboard/"
os.makedirs(tensorboard_log_dir, exist_ok=True)

n_training_envs = 4

# Define paths for saving models and evaluation results
left_model_path = "/content/drive/MyDrive/2A_PONG_ppo/left_ppo_10x20k"
right_model_path = "/content/drive/MyDrive/2A_PONG_ppo/right_ppo_10x20k"

  np.bool8: (False, True),

  from scipy.ndimage.filters import sobel



Mounted at /content/drive


In [4]:
class TensorboardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(TensorboardCallback, self).__init__(verbose)
        self.rewards = []

    def _on_step(self) -> bool:
        reward = self.locals['rewards'][0]  # assuming single environment (not vectorized)
        self.rewards.append(reward)

        if len(self.rewards) >= 100:
            mean_reward = np.mean(self.rewards[-100:])
            self.logger.record('mean_reward', mean_reward)

        return True

In [5]:
def train_PONG_supersuit(env_fn, steps: int = 20000, seed: int | None = 0, **env_kwargs):
    # Train a single model to play as each agent in a cooperative Parallel environment
    env = env_fn.parallel_env(**env_kwargs)
    env = ss.color_reduction_v0(env, mode='full')
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, 4)

    env.reset(seed = seed)

    eval_env = ss.pettingzoo_env_to_vec_env_v1(env)
    eval_env = ss.concat_vec_envs_v1(eval_env, n_training_envs, num_cpus=2, base_class="stable_baselines3")

    env = ss.pettingzoo_env_to_vec_env_v1(env)
    env = ss.concat_vec_envs_v1(env, n_training_envs, num_cpus=2, base_class="stable_baselines3")


    print(f"Starting training")

    # Load or create models for both agents
    if os.path.exists(left_model_path + ".zip") and os.path.exists(right_model_path + ".zip"):
        model_left = PPO.load(left_model_path, env=env)
        model_right = PPO.load(right_model_path, env=env)
    else:
        model_left = PPO("CnnPolicy", env, verbose=1, tensorboard_log=tensorboard_log_dir)
        model_right = PPO("CnnPolicy", env, verbose=1, tensorboard_log=tensorboard_log_dir)

    # Callback for saving models and logging to TensorBoard
    eval_callback_left = EvalCallback(eval_env, best_model_save_path=left_model_path,
                                      log_path=tensorboard_log_dir, eval_freq=5000,
                                      deterministic=True, render=False)

    eval_callback_right = EvalCallback(eval_env, best_model_save_path=right_model_path,
                                       log_path=tensorboard_log_dir, eval_freq=5000,
                                       deterministic=True, render=False)

    tensorboard_callback = TensorboardCallback()

    for i in range(10):  # 30 iterations of self-play training
        print(f"Self-play iteration {i+1}")

        # Train the left agent
        model_left.learn(total_timesteps=steps, callback=[eval_callback_left, tensorboard_callback])
        model_left.save(left_model_path)  # Save the left agent

        # Train the right agent
        model_right.learn(total_timesteps=steps, callback=[eval_callback_right, tensorboard_callback])
        model_right.save(right_model_path)  # Save the right agent

    print("Model has been saved.")
    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.")
    env.close()


In [6]:
def resize_frame(frame, macro_block_size=16):
    height, width, _ = frame.shape
    new_height = (height + macro_block_size - 1) // macro_block_size * macro_block_size
    new_width = (width + macro_block_size - 1) // macro_block_size * macro_block_size
    resized_frame = np.array(Image.fromarray(frame).resize((new_width, new_height)))
    return resized_frame

In [7]:
def eval_timesteps(env_fn, num_timesteps: int = 1000, render_mode: str | None = None, **env_kwargs):
    # Evaluate trained agents for a fixed number of timesteps
    env = env_fn.env(render_mode="rgb_array", **env_kwargs)
    env = ss.color_reduction_v0(env, mode='full')
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, 4)

    print(f"\nStarting evaluation")

    # Load the latest policies for both agents
    try:
        latest_policy_left = max(glob.glob("/content/drive/MyDrive/2A_PONG_ppo/left_ppo_10x20k.zip"))
    except ValueError:
        print("Policy agent 1 not found.")
        exit(0)

    try:
        latest_policy_right = max(glob.glob("/content/drive/MyDrive/2A_PONG_ppo/right_ppo_10x20k.zip"))
    except ValueError:
        print("Policy agent 2 not found.")
        exit(0)

    model_left = PPO.load(latest_policy_left)
    model_right = PPO.load(latest_policy_right)

    rewards = {agent: 0 for agent in env.possible_agents}

    frames = []

    # Run evaluation for a fixed number of timesteps
    env.reset()
    timestep = 0
    while timestep < num_timesteps:
        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()
            rewards[agent] += reward

            if termination or truncation:
                env.step(None)
            else:
                if agent == "first_0":  # Assuming the first agent's identifier
                    act = model_left.predict(obs, deterministic=True)[0]
                else:  # Assuming the second agent's identifier
                    act = model_right.predict(obs, deterministic=True)[0]
                env.step(act)

            if render_mode == "human":
                frame = env.render()
                resized_frame = resize_frame(frame)
                frames.append(resized_frame)

            timestep += 1
            if timestep >= num_timesteps:
                break
        if timestep >= num_timesteps:
            break

    env.close()

    avg_reward = sum(rewards.values()) / len(rewards.values())
    print("Rewards: ", rewards)
    print(f"Avg reward: {avg_reward}")

    if render_mode == "human":
        # Create a video from frames using moviepy
        clip = ImageSequenceClip(frames, fps=10)
        clip.write_videofile('/content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4')

        print("Evaluation saved as /content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4")
    return avg_reward


In [8]:
def show_video(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video width="480" height="320" controls><source src="{data_url}" type="video/mp4"></video>')


In [16]:
if __name__ == "__main__":
    display = Display(visible=0, size=(1400, 900))
    display.start()

    env_fn = pong_v3
    env_kwargs = {}

    # Train a model
    train_PONG_supersuit(env_fn, steps=20000, seed = 0, **env_kwargs)


  self.pid = os.fork()



Starting training
Using cpu device
Wrapping the env in a VecTransposeImage.
Using cpu device
Wrapping the env in a VecTransposeImage.
Self-play iteration 1
Logging to /content/drive/MyDrive/2A_PONG_ppo/tensorboard/PPO_2


  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")



------------------------------
| mean_reward        | -0.01 |
| time/              |       |
|    fps             | 302   |
|    iterations      | 1     |
|    time_elapsed    | 54    |
|    total_timesteps | 16384 |
------------------------------
-------------------------------------------
| mean_reward             | 0.0           |
| time/                   |               |
|    fps                  | 59            |
|    iterations           | 2             |
|    time_elapsed         | 551           |
|    total_timesteps      | 32768         |
| train/                  |               |
|    approx_kl            | 9.8806464e-05 |
|    clip_fraction        | 0.000757      |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.79         |
|    explained_variance   | -0.0024       |
|    learning_rate        | 0.0003        |
|    loss                 | 0.028         |
|    n_updates            | 10            |
|    policy_gradient_loss | 6.95e-05      |
|   

  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")



------------------------------
| mean_reward        | 0.0   |
| time/              |       |
|    fps             | 254   |
|    iterations      | 1     |
|    time_elapsed    | 64    |
|    total_timesteps | 16384 |
------------------------------
------------------------------------------
| mean_reward             | 0.0          |
| time/                   |              |
|    fps                  | 58           |
|    iterations           | 2            |
|    time_elapsed         | 560          |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0073927836 |
|    clip_fraction        | 0.0136       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.79        |
|    explained_variance   | -0.00316     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0344       |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000524    |
|    value_loss      

  warnings.warn(



Eval num_timesteps=7232, episode_reward=-4.20 +/- 20.58
Episode length: 2913.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 2.91e+03 |
|    mean_reward     | -4.2     |
| mean_reward        | 0.01     |
| time/              |          |
|    total_timesteps | 7232     |
---------------------------------
New best mean reward!
------------------------------
| mean_reward        | -0.01 |
| time/              |       |
|    fps             | 121   |
|    iterations      | 1     |
|    time_elapsed    | 135   |
|    total_timesteps | 16384 |
------------------------------
-----------------------------------------
| mean_reward             | 0.0         |
| time/                   |             |
|    fps                  | 51          |
|    iterations           | 2           |
|    time_elapsed         | 637         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.011

In [9]:
if __name__ == "__main__":
  display = Display(visible=0, size=(1400, 900))
  display.start()

  env_fn = pong_v3
  env_kwargs = {}
    # Evaluate 10 games (average reward should be positive but can vary significantly)
    #eval(env_fn, num_games=10, render_mode=None, **env_kwargs)
    # Watch 2 games
  eval_timesteps(env_fn, num_timesteps=1000, render_mode="human", **env_kwargs)
    # Display the video after evaluation
  video_path = "/content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4"
  display.stop()


Starting evaluation
Rewards:  {'first_0': -2, 'second_0': -1}
Avg reward: -1.5
Moviepy - Building video /content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4.
Moviepy - Writing video /content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4
Evaluation saved as /content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4


In [10]:
from IPython.display import HTML
from base64 import b64encode

# Convert mp4 to base64
video_path = "/content/drive/MyDrive/2A_PONG_ppo/video_10x20k.mp4"
video = open(video_path, "rb").read()
video_encoded = b64encode(video).decode()

# Display video
HTML(f"""
<video width="640" height="480" controls>
      <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
</video>
""")
